# Versionskontrolle

Dieses Jupyter Notebook enthält die Beispiele zur Vorlesung Versionsverwaltung mit Git. Um dieses Notebook auszuführen benötigt man einen [Bash-Kernel](https://github.com/takluyver/bash_kernel). Alle Kommandos in diesem Notebook simulieren die Ausführung an der Kommandozeile.

## Diff

Bevor wir uns Git zuwenden, müssen wir zunächst die Darstellung von Unterschieden zwischen Dateien betrachten. Man nennt so einen Unterschied einen *Diff*, und erstellt werden diese vom klassischen Werkzeug `diff`.

Gegeben seien zwei Beispieldateien (deren Inhalt wir uns mit `cat` ansehen können):

In [ ]:
cat data/git/Example1.java

In [ ]:
cat data/git/Example2.java

Der Unterschied zwischen den beiden Dateien kann mit `diff <datei1> <datei2>` angezeigt werden.

In [ ]:
diff data/git/Example1.java data/git/Example2.java

Angezeigt werden nur jene Zeilen, bei denen sich die beiden Dateien unterscheiden. Zeilen mit dem Prefix `<` enstammen der ersten Datei, und jene mit dem Prefix `>` sind die entsprechenden Zeilen in der zweiten Datei.

Sehen wir uns ein Beispiel an, bei dem neue Zeilen im Vergleich zur ersten Datei hinzugefügt wurden:

In [ ]:
cat data/git/Example3.java

In [ ]:
diff data/git/Example1.java data/git/Example3.java

Nun noch ein Beispiel bei dem Zeilen entfernt wurden.

In [ ]:
cat data/git/Example4.java

In [ ]:
diff data/git/Example1.java data/git/Example4.java

### Context

Um einen Diff besser zu verstehen, kann man sich den Kontext, also die Zeilen rund um die Änderung, mitanzeigen lassen.

In [ ]:
diff -c data/git/Example1.java data/git/Example2.java

In [ ]:
diff -c data/git/Example1.java data/git/Example3.java

In [ ]:
diff -c data/git/Example1.java data/git/Example4.java

### Unified diff format

Als Austauschformat wird das _Unified_-Diff-Format verwendet

In [ ]:
diff -u data/git/Example1.java data/git/Example2.java

Zeilen mit Präfix `+` wurden hinzugefügt, mit Präfix `-` entfernt.

In [ ]:
diff -u data/git/Example1.java data/git/Example3.java

In [ ]:
diff -u data/git/Example1.java data/git/Example4.java

## Patch

Das Unified Diff Format erlaubt es, bestehende Dateien anzupassen. Der Unterschied wird dazu in einem *Patch* gespeichert.

In [ ]:
diff -u data/git/Example1.java data/git/Example2.java > patch.txt

In [ ]:
cat patch.txt

In [ ]:
cp data/git/Example1.java Example.java

In [ ]:
cat Example.java

Um die Datei `Example.java` nun basierend auf unserem Patch zu verändern, benötigen wir das Kommando `patch`.

In [ ]:
patch -p0 Example.java < patch.txt

In [ ]:
cat Example.java

In [ ]:
# Nur um das Jupyter Notebook aufzuräumen...
rm patch.txt
rm Example.java

## Git

Wir verwenden zur Versionsverwaltung das Programm `git`. Git ist in alle modernen IDEs direkt integriert, wir verwenden hier jedoch die Kommandozeilen-Version.

Zunächst legen wir uns nur ein temporäres Verzeichnis an, damit das Jupyter Notebook nicht zugemüllt wird...

In [ ]:
rm -rf tmp
mkdir -p tmp
cd tmp

Wir nehmen an Entwickler 1 hat einen Workspace im Verzeichnis `dev1.

In [ ]:
mkdir -p dev1
cd dev1

Wir legen hier nun ein neues Git Repository an.

In [ ]:
git init

Um Veränderungen im Workspace zu simulieren, fügen wir eine Datei `hello.txt` hinzu.

In [ ]:
echo "Hallo SE 2024" > hello.txt

In [ ]:
ls

Um diese neue Datei in die Versionsverwaltung aufzunehmen, dient das `add` Kommando in Git.

In [ ]:
git add hello.txt

Damit befindet sich die Datei auf der "Stage". Um sie ins Repository einzuchecken, dient das `commit` Kommando.

In [ ]:
git commit -m "Add hello"

Git-Output wird mit einem "Pager" angezeigt, der per Default auf einen Tastendruck wartet. Nachdem das im Jupyter Notebook nicht geht, müssen wir den an dieser Stelle umkonfigurieren. Das ist normalerweise nicht notwendig.

In [ ]:
# Only required for presenting in the Jupyter notebook
export GIT_PAGER=cat

Die Historie unserers Repositories können wir mit `log` ansehen.

In [ ]:
git log

Nun verändern wir die Datei nochmal (und simulieren damit, dass Entwickler 1 arbeitet).

In [ ]:
echo "Hello again" >> hello.txt

Den Status unseres Workspaces können wir mit `status` einsehen.

In [ ]:
git status

Wenn wir wissen wollen, was verändert wurde, können wir uns einen Diff anzeigen lassen.

In [ ]:
git diff

Das Committen der neuen Änderungen funktioniert wieder ähnlich.

In [ ]:
git add hello.txt

In [ ]:
git commit -m "Updated hello"

Die Historie zeigt uns nun zwei Änderungen.

In [ ]:
git log

Wenn wir wissen wollen, wer zuletzt die Datei `hello.txt` geändert hat, dann finden wir das mit `blame` heraus.

In [ ]:
git blame hello.txt

### Nicht-committete Änderungen rückgängig machen

Angenommen wir nehmen wieder eine lokale Änderung vor.

In [ ]:
echo "Hello yet again" >> hello.txt

In [ ]:
git status

Wenn wir lokale Änderungen, die noch nicht "staged" oder "committed" sind, rückgängig machen wollen, dann können wir dazu einfach die Version in `HEAD` auschecken.

In [ ]:
git checkout hello.txt

In [ ]:
git status

In [ ]:
cat hello.txt

### Unstaging

Nehmen wir noch eine Änderung vor:

In [ ]:
echo "Hello yet again" >> hello.txt
git add hello.txt

In [ ]:
git status

Diese Änderung ist nun schon "staged". Wenn wir diese Änderung rückgängig machen wollen, reicht ein "checkout" nicht aus:

In [ ]:
git checkout hello.txt

In [ ]:
git status

Zuvor müssen wir die Datei "unstagen":

In [ ]:
git reset hello.txt

In [ ]:
git status

Aber committen wir die Änderung zunächst doch noch.

In [ ]:
git add hello.txt
git commit -m "Hello the third"

### Reverting

Wir können eine beliebige Version der Datei `hello.txt` auschecken. Die folgenden Commits existieren:

In [ ]:
git log

Da sich Commit-Hashes bei jeder Ausführung ändern, holen wir uns den Commithash des ersten Commits (das ist normal nicht so notwendig, das passiert hier nur um mit dem Jupyter Notebook zu arbeiten).

In [ ]:
# Normally you'd know the commit hash, we need to figure out this way for the Jupyter notebook
COMMIT=$(git log --reverse --oneline | head -n 1 | cut -d ' ' -f1)

Wir können uns nun gezielt die Version von `hello.txt` mit diesem Commithash auschecken.

In [ ]:
git checkout $COMMIT -- hello.txt

In [ ]:
git status

In [ ]:
cat hello.txt

Die Datei zählt nun als geändert, wenn wir diese Änderung behalten wollen (und damit die letzten beiden Commits rückgängig machen), dann müssen wir sie commiten.

In [ ]:
git add hello.txt
git commit -m "Reverted to original version"

In [ ]:
git log hello.txt

### Detached Head

Vorsichtig muss man sein, wenn man nicht nur eine Datei in einer alten Version auscheckt, sondern den ganzen Workspace in den Zustand versetzt.

In [ ]:
git checkout $COMMIT

Wenn wir uns die Historie ansehen, sind die letzten beiden Commits nun verschwunden:

In [ ]:
git log --oneline

Noch ist nichts verloren, wir können einfach wieder die "main" Version auschecken.

In [ ]:
git checkout main

In [ ]:
git log --oneline

`main` ist ein Branch. Um zu sehen, wie wir dieses Problem umgehen können, schauen wir uns zunächst an wie man mit Branches arbeitet.

## Branches

Zwischen Branches wechselt man mit dem Kommando `checkout`; einen neuen Branch erstellt man mit `checkout -b`.

In [ ]:
git checkout -b se2

Nun können wir in unserem `se2` Branch arbeiten.

In [ ]:
echo "Hello in the se2 branch" >> hello.txt

In [ ]:
git add hello.txt

In [ ]:
git commit -m "Add branch hello"

In [ ]:
git log --oneline

Wenn wir zurück in den `main` Branch wechseln, dann ist die letzte Änderung hier nicht vorhanden.

In [ ]:
git checkout main

In [ ]:
cat hello.txt

In [ ]:
git log --oneline

Um Branches zusammenzuführen, dient das Kommando "merge". Mergen wir also `se2` nach `main`:

In [ ]:
git merge se2

Nun sind die Änderungen wieder zusammengeführt.

In [ ]:
cat hello.txt

Änderungen in main sind in `se2` nicht sichtbar:

In [ ]:
echo "Hello added in main" >> hello.txt
git add hello.txt
git commit -m "Added hello main"

In [ ]:
git checkout se2

In [ ]:
cat hello.txt

Wir führen nun auch in `se2` eine Änderung durch und committen.

In [ ]:
echo "Hello added again in se2" >> hello.txt
git add hello.txt
git commit -m "Added hello to se2 again"

Diese Änderung ist natürlich in `main` nicht sichtbar.

In [ ]:
git checkout main

In [ ]:
cat hello.txt

Wenn wir nun allerdings einen Merge probieren, gibt es ein Problem: Die Datei `hello.txt` wurde in beiden Branches editiert.

In [ ]:
git merge se2

Der Merge-Conflict wird in der Datei angezeigt.

In [ ]:
cat hello.txt

In [ ]:
git status

Hier hat der automatische Merge nicht geklappt. Wir müssen also per Hand in unserem Editor das Problem lösen.

In [ ]:
cat > hello.txt << EOM
Hallo SE 2022
Hello in the se2 branch
Hello added in main
Hello added again in se2
EOM

In [ ]:
cat hello.txt

Wir befinden uns noch im Merge, und müssen diesen abschliessen indem wir die Datei, in der wir den Merge-Conflict behoben haben, stagen und dann committen.

In [ ]:
git status

In [ ]:
git add hello.txt

Ein Merge-Commit braucht keine explizite Commit-Message.

In [ ]:
git commit --no-edit

In [ ]:
git log --oneline

Wir können uns hierzu auch einen Commit-Graphen anzeigen lassen.

In [ ]:
git log --oneline --graph

### Detached Head reparieren

Erstellen wir nochmal den problematischen Detached Head:

In [ ]:
# Checkout first commit
git checkout $COMMIT

Die Lösung des Problems liegt darin, für den aktuellen Zustand einen neuen Branch anzulegen.

In [ ]:
git checkout -b new_feature

In [ ]:
echo "Adding hello to initial hello" >> hello.txt

In [ ]:
git add hello.txt

In [ ]:
git commit -m "Added hello in branch"

In [ ]:
git log --oneline --graph

Diese Änderung ist natürlich noch nicht im Main-Branch:

In [ ]:
git checkout main

In [ ]:
cat hello.txt

## Rebase

Ein Merge erzeugt einen expliziten Merge-Commit und eine Verzweigung in der Commit-Historie. Eine Alternative dazu ist ein Rebase, welches die Commits eines Branches auf einen alternativen Startpunkt anwendet. Wir erzeugen uns dazu wieder einen Branch der von dem ersten Commit verzweigt.

In [ ]:
git checkout $COMMIT

In [ ]:
git checkout -b rebase_feature

In diesem Branch erzeugen wir eine neue Datei `hello2.txt`.

In [ ]:
echo "Hello again" > hello2.txt

In [ ]:
git add hello2.txt
git commit -m "Add new hello file"

In diesem Branch fehlen uns nun aber die ganzen Änderungen, die an der Datei `hello.txt` angwendet wurden:

In [ ]:
cat hello.txt

In [ ]:
git log --oneline --graph

Wir können den Branch `rebase_feature` aber mit dem aktuellen Main rebasen:

In [ ]:
git rebase main

Hierdurch wird der Commit, mit dem `hello2.txt` erstellt wurde, auf den Main angewendet.

In [ ]:
cat hello.txt

In [ ]:
git log --oneline --graph

Die Datei `hello2.txt` existiert aber noch nicht im Main:

In [ ]:
git checkout main

In [ ]:
ls

Wir müssen den Branch dazu nach Main mergen.

In [ ]:
git merge rebase_feature

In [ ]:
ls

Nachdem der Branch `rebase_feature` auf Main rebased war, wird ein Fast-Forward-Merge angewendet, d.h. es benötigt keinen eigenen Merge-Commit.

In [ ]:
git log --oneline --graph

## Stashing

Es kann vorkommen, dass man Änderungen von anderer Stelle einpflegen muss, bevor das Feature, an dem man gerade arbeitet, fertig ist, sodass man es noch nicht committen will. Man kann die Änderungen, die noch nicht committed sind, auf einen "Stash" schieben.

In [ ]:
echo "Some unfinished change" >> hello.txt

In [ ]:
cat hello.txt

In [ ]:
git status

In [ ]:
git stash

Damit sind nun alle Dateien wieder am gleichen Stand wie `master`.

In [ ]:
git status

In [ ]:
cat hello.txt

Um die lokalen Änderungen wieder einzufügen, verwendet man `stash apply`:

In [ ]:
git stash apply

In [ ]:
cat hello.txt

In [ ]:
# Houskeeping for Jupyter notebook
cd ..
rm -rf dev1

## Mehrere Benutzer und Remote Repositories

Git ist ein verteiles Versionskontrollsystem, d.h. es kann mehrere Klone des Repositories geben, und man kann Änderungen dazwischen austauschen. Wir legen uns für das Notebook ein neues "Remote" Repository an, auf welches 2 simulierte Entwickler Zugriff haben sollen:

In [ ]:
mkdir remote_repo
cd remote_repo
git init --bare

Entwickler 1 und Entwickler 2 können sich Klone des Repositories mit dem "clone" Befehl erstellen. (Der Befehl wird aktuell noch meckern dass das Repository leer ist).

In [ ]:
cd ..

In [ ]:
git clone remote_repo dev1

In [ ]:
git clone remote_repo dev2

Zunächst simulieren wir, dass Entwickler 1 in seinem Workspace arbeitet.

In [ ]:
cd dev1

In [ ]:
echo "I am developer 1" >> hello.txt
git add hello.txt
git commit -m "Hello 1"

Um die Änderungen im eigenen Repository an das "Remote" Repository zu schieben, dient der "push" Befehl:

In [ ]:
git push

Entwickler 2 kann nun diese Änderungen vom Remote Repository in das eigene "pullen":

In [ ]:
cd ..
cd dev2

Noch ist das Repository leer:

In [ ]:
ls

Änderungen ziehen passiert mit "pull":

In [ ]:
git pull

In [ ]:
cat hello.txt

Nun arbeitet auch Entwickler 2 im eigenen Workspace.

In [ ]:
echo "I am developer 2" >> hello.txt
git add hello.txt
git commit -m "Add hello from developer 2"

Die Änderungen können per "push" wieder geteilt werden.

In [ ]:
git push

Entwickler 1 kann sich diese Änderungen mit "pull" holen.

In [ ]:
cd ..
cd dev1

In [ ]:
cat hello.txt

In [ ]:
git pull

In [ ]:
git log

In [ ]:
cat hello.txt

In [ ]:
cd ..

## Remotes

Ein Git Repository ist nicht an ein einzelnes Remote-Repository gebunden, sondern kann sich mit beliebig vielen anderen Repositories austauschen. Klonen wir zunächst ein Repository von GitHub:

In [ ]:
git clone https://github.com/se2p/se2022-gitexample.git

In [ ]:
cd se2022-gitexample

Wenn ein Repository per "clone" erstellt wird, dann heisst das Upstream-Repository `origin`. Wir können uns ansehen, wo `origin` liegt:

In [ ]:
git remote get-url origin

Legen wir uns noch einen Klon des neuen Repositories an:

In [ ]:
cd ..
git clone se2022-gitexample another_clone

In [ ]:
cd another_clone

`origin` ist nun einfach das Verzeichnis unseres lokalen Git-Repositories, das wir eben geklont haben:

In [ ]:
git remote get-url origin

Wir können aber nun auch das GitHub-Repository als Remote hinzufügen:

In [ ]:
git remote add github https://github.com/se2p/se2022-gitexample.git

Unser Repository hat nun zwei Remotes:

In [ ]:
git remote

Der Name des Remotes kann bei einem `push` angegeben werden.

### Hinweis zum Pushen von Branches an Remotes

Befindet man sich in einem Branch, der auf einem Remote noch nicht existiert, so resultiert ein `git push` in der folgenden Fehlermeldung:

```
fatal: The current branch Foo has no upstream branch.
To push the current branch and set the remote as upstream, use

    git push --set-upstream origin <branchname>
```

Durch Eingabe des vorgegebenen Befehls `git push --set-upstream origin <branchname>` wird der Branch am Remote `origin` angelegt. Danach kann man Dateiänderungen direkt in den Remote-Branch pushen. Alternativ erlaubt auch `git push --all` dass _alle_ Branches gepushed werden.

### Hinweis zum Pushen von Tags an Remotes

Wenn nicht nur Dateiänderungen sondern auch Tags gepushed werden sollen, so muss man den Tagnamen der gepushed werden soll angeben: `git push origin <tagname>`. Alternativ pushed `git push --follow-tags` einfach alle Tags.


In [ ]:
# Clean up notebook
cd ../..
rm -rf tmp